# EVmutation Supplementary Code

*Important note*: This is the initial version of the code distributed with the publication. Please check back regularly on http://evmutation.org and http://github.com/debbiemarkslab for the most up-to-date versions of the code and new features.

This HTML file is the static version of the included Jupyter notebook (EVmutation.ipynb) that demonstrates usage of the Python code for mutation effect prediction.

## 1. Installation instructions

### plmc

C code to infer pairwise undirected graphical models for families of biological sequences. For installation instructions, please refer to README.md in the plmc subdirectory.

Github repository: http://github.com/debbiemarkslab/plmc

### Python code

Python code to compute mutation effects from a graphical model inferred using plmc. This code requires an up-to-date Python 3 installation, and the following packages (we recommend using the latest Anaconda Python 3 distribution from https://www.continuum.io, which includes all of these packages by default): 

* numpy
* scipy
* pandas
* numba

Github repository: http://github.com/debbiemarkslab/EVmutation


## 2. Computing pairwise models of sequences

### Alignment requirements

The first step of the EVmutation method is to compute a pairwise model of sequences from a family sequence alignment, e.g. the included example file (example/PABP_YEAST.a2m). This alignment has to be in aligned FASTA/A2M format and must fulfill the following requirements:

* The target sequence may not contain any gaps.
* Columns that should be excluded from model inference (e.g. too many gaps) must be represented by lowercase characters. Gaps in these positions must be represented by a dot (".") rather than a dash ("-").
* The identifier of the target sequence has to be passed to plmc with the -f parameter (see below)

### Regularization

We adjust the strength of $l_2$-regularization $\lambda_J$ on the couplings based on the length of the model that we infer as $0.2 * (N - 1)$, where $N$ is the length of the model. $N$ corresponds to the number of uppercase positions in the alignment file. For the included PABP_YEAST example, there are 82 uppercase residues in the PABP_YEAST sequence, which means that we choose a regularization strength of $\lambda_J = (82-1) * 0.2 = 16.2$. This value is passed to plmc using the -le command line option.


### Running plmc

To infer model parameters from the alignment, please run

```bash
plmc/bin/plmc -o example/PABP_YEAST.model_params -c example/PABP_YEAST.txt -f PABP_YEAST -le 16.2 -lh 0.01 -m 200 -t 0.2 -g example/PABP_YEAST.a2m
```

This will generate a binary file with the model parameters (example/PABP_YEAST.model_params) and a text file with the summarized epistatic constraint between pairs of positions (PABP_YEAST.txt). The model parameters can then be used to predict the effects of mutations.

Depending on the input alignment, it may be necessary to adjust the sequence alphabet (e.g. for RNA sequences using ```-a "-ACGU"```) or the identity cutoff for sequence reweighting (```-t```, e.g. for viral proteins). Additional information about the meaning of the different command line parameters can be obtained using ```plmc/bin/plmc -h```.

## 3. Predicting mutation effects

In [1]:
import pandas as pd

from model import CouplingsModel
import tools

In [2]:
# load parameters from file to create a pairwise model
c = CouplingsModel("example/PABP_YEAST.model_params")

### Predicting an experimental dataset

In [3]:
# read the experimental mutational scanning dataset for PABP by Melamed et al., RNA, 2013
data = pd.read_csv(
    "example/PABP_YEAST_Fields2013-singles.csv", sep=";", comment="#"
)

# predict mutations using our model
data_pred = tools.predict_mutation_table(
    c, data, "effect_prediction_epistatic"
)

In [4]:
# can also add predictions by the corresponding independent model
c0 = c.to_independent_model()

data_pred = tools.predict_mutation_table(
    c0, data_pred, "effect_prediction_independent"
)

In [5]:
data_pred.head()

,mutant,linear,log,effect_prediction_epistatic,effect_prediction_independent
0,G126A,0.711743,-0.490571,-2.610615,0.406487
1,G126C,0.449027,-1.155127,-5.663638,-0.027602
2,G126E,0.588928,-0.763836,-6.611062,-1.827570
3,G126D,0.229853,-2.121218,-7.270577,-1.180076
4,G126N,0.679435,-0.557593,-5.809167,0.387440


### Predicting single-substitution landscape (independent of experiment)

In [6]:
singles = tools.single_mutant_matrix(
    c, output_column="effect_prediction_epistatic"
)

singles.head()

,mutant,pos,wt,subs,frequency,effect_prediction_epistatic
0,K123A,123,K,A,0.077201,0.796669
1,K123C,123,K,C,0.001461,-3.337328
2,K123D,123,K,D,0.118235,-0.316713
3,K123E,123,K,E,0.110503,-1.078200
4,K123F,123,K,F,0.007791,-3.013274


### Predicting arbitrary mutations

In [7]:
# double mutant L186M, G188A
delta_E, delta_E_couplings, delta_E_fields = c.delta_hamiltonian([(186, "L", "M"), (188, "G", "A")])
delta_E

-6.9225586684769951